# Decision Tree Classifier
---
This notebook on Decision Tree Algorithm in Python will take you through the fundamentals of `Decision Tree` machine learning algorithm concepts and its demo in Python

> ## Import Required Libraries

In [17]:
import numpy as np
import pandas as pd

> ## Creating the dataset
In the following data
- Each row is a sample
- The last column is the label
- The first two columns are independent features
- Interesting note: 2nd and 5th examples have the same features, but different labels

Let's see how tree handles this case

In [18]:
training_data = [
                 ['Green', 3, 'Mango'],
                 ['Yellow', 3, 'Mango'],
                 ['Red', 1, 'Grape'],
                 ['Red', 1, 'Grape'],
                 ['Yellow', 3, 'Lemon']
]

> ### Column labels
These are used to print the tree

In [19]:
header = ['Color', 'Diameter', 'Label']

> ## Write a function to find the unique values for a column in a data set

In [20]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

> ### Unique_vals demo

In [21]:
unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

In [22]:
unique_vals(training_data, 1)

{1, 3}

> ## Write a function to count the no.of each type of sample in the dataset

In [23]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

> ### class_counts demo

In [24]:
class_counts(training_data)

{'Grape': 2, 'Lemon': 1, 'Mango': 2}

> ## Write a function to check a given value is numeric or not

In [25]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

> ### is_numeric demo

In [26]:
is_numeric(7)

True

In [27]:
is_numeric('Red')

False

> ## Define a class partitions the data set
This class just records a `column number` (e.g., 0 for color) and a `column value` (e.g., Green). The `match` method is used to compare the feature value in a sample to the feature value stored in the question. 

In [34]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, sample):
        """ Compares the feature value in a sample to the feature value in this question """
        val = sample[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        """ This is just a helper method to print the question in a readable format """
        condition = '=='
        if is_numeric(self.value):
            condition = '>='
        return 'Is %s %s %s?' % ( header[self.column], condition, str(self.value))

    

In [44]:
def partition(rows, question):
    """ Partitions the data set 
    For each row in the dataset, check if it matches the question. If so, add it to 'true rows', otherwise, add it to 'false rows' """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def gini(rows):
    """ Calculate the Gini Impurity for a list of rows """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl ** 2
    return impurity        

def info_gain(left, right, current_uncertainty):
    """ Information Gain 
    The uncertainity of the starting node, minus the weighted impurity of two child nodes """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1-p) * gini(right)

def find_best_split(rows):
    """ Find the best question to ask by iterating over every feature/value and calculating the information gain """
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

In [47]:
class leaf:
    """ A leaf node classifies data 
    This holds a dictionary of class (ex. 'Mango') -> no.of times it appears 
    in the rows from the training data that reach this leaf """
    def __init__(self, rows):
        self.predictions = class_counts(rows)

class Decision_Node:
    """ A Decision node asks a question
    This holds a reference to the question, and to the two child nodes """
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

def build_tree(rows):
    """ Builds the tree
    Rules of recursion: 1) Believe that it works 2) Start by checking for the base case 
    (no further information gain) 3) Prepare for """
    gain, question = find_best_split(rows)
    if gain == 0:
        return leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

def print_tree(node, spacing=''):
    if isinstance(node, leaf):
        print(spacing + 'Predict', node.predictions)
        return
    print(spacing + str(node.question))
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + '  ')
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + '  ')

def classify(row, node):
    if isinstance(node, leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + '%'
    return probs

In [49]:
if __name__ == '__main__':
    my_tree = build_tree(training_data)
    print_tree(my_tree)
    testing_data = [
                    ['Green', 3, 'Mango'],
                    ['Yellow', 4, 'Mango'],
                    ['Red', 2, 'Grape'],
                    ['Red', 1, 'Grape'],
                    ['Yellow', 3, 'Lemon']
    ]
    for row in testing_data:
        print('Actual: %s. Predicted: %s' % (row[-1], print_leaf(classify(row, my_tree))))

Is Diameter >= 3?
--> True:
  Is Color == Yellow?
  --> True:
    Predict {'Mango': 1, 'Lemon': 1}
  --> False:
    Predict {'Mango': 1}
--> False:
  Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
